# Lab-8- XML Queries In Python
## CSCI 3287 <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right">

### Enter your name here: Mitch Henderson
#### Name: Mitch Henderson
#### Identikey: mihe2568
#### Hours 12

If you haven't already reviewed the LXML tutorial and played with the LXML notebook, you're encouraged to do so.

We're going to use to [lxml](https://lxml.de/) library to access and manipulate XML data. Some of this material is an adaptation of the [lxml tutorial](https://lxml.de/tutorial.html).

The LXML library doesn't provide an XQuery interface, so we're going to use a tool called XQilla as a commandline interface.


In [43]:
import lxml
import lxml.etree as etree
import lxml.html

All of the problems (XPath, Xquery, XSLT) will use the **products** XML document below. I've created an LXML ElementTree for that document. I've also created a copy of the document in the file `/tmp/products.xml` that can be used for the XQuery exercise.

In [44]:
products_text = """<Products>
  <Maker name="A">
    <PC model="1001" price="2114">
      <Speed>2.66</Speed>
      <RAM>1024</RAM>
      <HardDisk>250</HardDisk>
    </PC>
    <PC model="1002" price="995">
      <Speed>2.10</Speed>
      <RAM>512</RAM>
      <HardDisk>250</HardDisk>
    </PC>
    <Laptop model="2004" price="1150">
      <Speed>2.00</Speed>
      <RAM>512</RAM>
      <HardDisk>60</HardDisk>
      <Screen>13.3</Screen>
    </Laptop>
    <Laptop model="2005" price="2500">
      <Speed>2.16</Speed>
      <RAM>1024</RAM>
      <HardDisk>120</HardDisk>
      <Screen>17.0</Screen>
    </Laptop>
  </Maker>
  <Maker name="E">
    <PC model="1011" price="959">
      <Speed>1.86</Speed>
      <RAM>2048</RAM>
      <HardDisk>160</HardDisk>
    </PC>
    <PC model="1012" price="649">
      <Speed>2.80</Speed>
      <RAM>1024</RAM>
      <HardDisk>160</HardDisk>
    </PC>
    <Laptop model="2001" price="3673">
      <Speed>2.00</Speed>
      <RAM>2048</RAM>
      <HardDisk>240</HardDisk>
      <Screen>20.1</Screen>
    </Laptop>
    <Printer model="3002" price="239">
      <Color>false</Color>
      <Type>laser</Type>
    </Printer>
  </Maker>
  <Maker name="H">
    <Printer model="3006" price="100">
      <Color>true</Color>
      <Type>ink-jet</Type>
    </Printer>
    <Printer model="3007" price="200">
      <Color>true</Color>
      <Type>laser</Type>
    </Printer>
  </Maker>
</Products>
"""

In [45]:
products = etree.XML(products_text)

with open("products.xml","w") as f:
    f.write(products_text)

## 1) XPath [ 45 points total ] 

To do the XPath homework, you'll need to use the Product XML file.

#### 1a) Find the amount of RAM on each PC [ 5 pts ]

Your output should look like `['1024', '512', '2048', '1024']`

In [46]:
### BEGIN SOLUTION
# // means any descendant of the current node
# in this case we are looking for any PC's
# text() extracts the text node from RAM element
products.xpath('//PC//RAM/text()')
### END SOLUTION

['1024', '512', '2048', '1024']

#### 1b) Find the price of each product of any kind. [ 5 pts ]

Your solution doesn't need to restrict itself to a PC or Printer -- it should report the price of *any* product.

In [47]:
### BEGIN SOLUTION
# // says any descendant of an element
# in this case we are returning the price of any element that
# has price as an attribute
products.xpath('//@price')
### END SOLUTION

['2114', '995', '1150', '2500', '959', '649', '3673', '239', '100', '200']

#### 1c) Find the text of all the printer elements. [5 pts ]

This should reduce the text of each element within a **Printer** element (e.g. 'false', 'laser', *etc*)

In [48]:
### BEGIN SOLUTION
# // any descendant 
# Printer some printer descendant
# /*/ any child element of Printer
# return the text() node of that element
products.xpath("//Printer/*/text()")
### END SOLUTION

['false', 'laser', 'true', 'ink-jet', 'true', 'laser']

#### 1d) Find the makers of laser printers. [ 5 pts ]

There are two makers of printers, 'E' and 'H'

In [49]:
### BEGIN SOLUTION
# // any descendant
# that is a Printer element
# look for the parent of the Printer node -> this should be a maker
# return the maker's name attribute
products.xpath('//Printer/parent::node()/@name')
### END SOLUTION

['E', 'H']

#### 1e) Find the makers of PC’s and/or laptops. [ 5 pts ]

There are two makers of PC and Laptops. Although your query could check for just one of PC or Laptop and be correct, you must construct your query to work even if one Maker only makes one or the other kind of product.

In [50]:
### BEGIN SOLUTION
# // descendant
# of Printer or Laptop
# Get the parent of a Printer or a Laptop
# This should be a Maker
# Then return the name attribute of the Maker
products.xpath('(//Printer | //Laptop)/parent::node()/@name')
### END SOLUTION

['A', 'E', 'H']

#### 1f) Find the model numbers of PC’s with a hard disk of at least 200 gigabytes [ 10 pts ]

In [51]:
### BEGIN SOLUTION
# // descendant
# For PC element
# Its HardDisk child
# [] with the condition
# That its text node is greater than 200
# Going back to the parent -> which should be the HardDisk
# and the model attribute for that PC
products.xpath('//PC/HardDisk[text() >= 200]/parent::node()/@model')
### END SOLUTION

['1001', '1002']

#### 1g) Find the makers that make at least two PC's [ 10 pts ]

This means that there are two PC's that are *siblings* in the element tree.

In [52]:
### BEGIN SOLUTION
# // descendant
# a Make node
# with a condition []
# such that there are at least two PC children, and get the Maker's name attribute
products.xpath('//Maker[count(PC)>=2]/@name')
### END SOLUTION

['A', 'E']

## XQuery [ 45 Pts Total ]

As mentioned, we need a completely different library and interface for XQuery and the [Python Binding for that library](https://pypi.org/project/python-simplexquery/) is not very robust.

So, we're using to use the command line tool "[xqilla](http://xqilla.sourceforge.net/HomePage)" to run our queries.

Let's use an [XQuery tutorial](https://www.w3schools.com/xml/xquery_intro.asp) that appears at the W3Schools website. That tutorial uses a **books.xml** dataset. As before, we'll include here as a string so you can see dataset.

#### 2a) Find the Printers models with a price less than 1000. [ 5 pts ]

Your output should look like `['3002', '3006', '3007']`

In [53]:
### BEGIN SOLUTION
# // descendant
# Printer element
# with condition [] that its price attribute is less than 1000
# get the model attribute
# and just the string() part of the value part of the model
with open("query.xq","w") as q:
    q.write('''//Printer[@price < 1000]/@model/string()''')
%system xqilla -i products.xml query.xq

### END SOLUTION

['3002', '3006', '3007']

#### 2b) Find the Printer...[ 5 pts ]

Find the Printer elements with a price <=200, and produce the sequence of the elements of these printers surrounded by a tag `<CheapPrinters>`

Your output should look like
```
['<CheapPrinters><Printer model="3006" price="100">',
 '      <Color>true</Color>',
 '      <Type>ink-jet</Type>',
 '    </Printer><Printer model="3007" price="200">',
 '      <Color>true</Color>',
 '      <Type>laser</Type>',
 '    </Printer></CheapPrinters>']
 ```

In [54]:
### BEGIN SOLUTION
# // descendant 
# Printer element
# [] conditioned
# price attribute <= 200
# {} xpath containers
# templated with <element></element>
with open("query.xq","w") as q:
    q.write('''<CheapPrinters>{//Printer[@price<=200]}</CheapPrinters>''')
### END SOLUTION
%system xqilla -i products.xml query.xq

['<CheapPrinters><Printer model="3006" price="100">',
 '      <Color>true</Color>',
 '      <Type>ink-jet</Type>',
 '    </Printer><Printer model="3007" price="200">',
 '      <Color>true</Color>',
 '      <Type>laser</Type>',
 '    </Printer></CheapPrinters>']

#### 2c) Find the names of the makers of both printers and laptops. [ 5 pts ]

You may want to read about quantification in XQuery

In [55]:
### BEGIN SOLUTION
# // descendants
# Printer | Laptop : Printer of Laptop element
# parent::node() : parent of the element
# get parent's name attribute
# string() extract the value part of the attribute
# distinct-values(): only return the distinct names
with open("query.xq","w") as q:
    q.write('''distinct-values(//(Printer|Laptop)/parent::node()/@name/string())''')
### END SOLUTION
%system xqilla -i products.xml query.xq

['A', 'E', 'H']

#### 2d) Find the names of the makers that produce at least two PC’s with a speed of 2.00 or more. [ 10 pts ]

There's only one such maker (`A`). This problem exercises the Axes.

In [56]:
### BEGIN SOLUTION
# //Maker for each Maker descendant
# for each $maker
# if matching where <condition>
# count() : count the number of elements $maker/PC
# []: satisfying a particular condition
#     where child element Speed has text() node value >= 2.0
# then return the name attribute value of the $maker
with open("query.xq","w") as q:
    q.write('''for $maker in //Maker
    where count($maker/PC[Speed/text() >= 2.0]) >= 2
    return $maker/@name/string()''')
### END SOLUTION
%system xqilla -i products.xml query.xq

['A']

#### 2e) Find the makers such that every PC they produce has a price no more than 1000. [ 10 pts ]

Note that a maker that does not make PC's might statisfy a constraint that every PC make by that maker is less than $1000....

In [57]:
### BEGIN SOLUTION
# loop over Maker elements
# where <condition> is satisfied
# count() : count of $maker/PC elements
# [] : with condition matching the PC's price attribute value <= 1000
# if that count() = count(all PC) : in other words, if $maker's every PC satisfies the [] condition
# then return the name attribute of the $maker
# string() : pulls out the value of the attribute
with open("query.xq","w") as q:
    q.write('''for $maker in //Maker
    where count($maker/PC[@price <= 1000]) = count($maker/PC)
    return $maker/@name/string()''')

### END SOLUTION
%system xqilla -i products.xml query.xq

['E', 'H']

#### 2f) Produce a sequence of elements of the form.. [ 10 pts ]

`<Laptop> <Model>x</Model> <Maker>y</Maker> </Laptop>`
where x is the model number and y is the name of the maker of the laptop.

Your output should look like
```
['<Laptop><Model>2004</Model><Maker>A</Maker></Laptop>',
 '<Laptop><Model>2005</Model><Maker>A</Maker></Laptop>',
 '<Laptop><Model>2001</Model><Maker>E</Maker></Laptop>']
```

In [58]:
### BEGIN SOLUTION
# for each //Laptop : laptop descendant
# return template with <Laptop> element (top)
# having the child elements <Model> and <Maker>
# we pull the model name attribute with @model/string()
# and the maker'sname by traversing up to the parent::node()
# and retrieving the parent's name attribute with @name/string()
with open("query.xq","w") as q:
    q.write('''
    for $laptop in //Laptop
        return <Laptop><Model>{$laptop/@model/string()}</Model><Maker>{$laptop/parent::node()/@name/string()}</Maker></Laptop>
''')
### END SOLUTION
%system xqilla -i products.xml query.xq

['<Laptop><Model>2004</Model><Maker>A</Maker></Laptop>',
 '<Laptop><Model>2005</Model><Maker>A</Maker></Laptop>',
 '<Laptop><Model>2001</Model><Maker>E</Maker></Laptop>']

## XSLT [ 10 pts total ]

You'll use LXML to perform the XSLT transformations. Most of the problems will ask you to present e.g. an HTML table. Rather than just look at the HTML, we'll use some Jupyter notebook tricks to enable us to display formatted HTML.

In [59]:
from IPython.core.display import display, HTML

/tmp/ipykernel_151/2058709175.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [60]:
table_data = '''<table> 
             <tr> <th> A Table </th> <th> w/2 headers </th> </tr>
             <tr> <td> Some Data </td> <td> More Data </td> </tr>
             </table>'''
display(HTML(table_data))

A Table,w/2 headers
Some Data,More Data


### 3a) An HTML Table [ 5 pts ]

The table should have a header “Manufacturers” followed by an enumerated list of the names of all the makers of products listed in the input.

In [61]:
### BEGIN SOLUTION
### Your code here
# The <template> element will generate elements that will be inserted
# into the DOM
#
# It can take a matching xpath, in this case we take /Product element
#
#Inside the template we use a for-each element
# which will select each Maker child of products
# and take the value-of the Maker's name attribute

p3a_xslt = etree.XML('''
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
<xsl:output method="text"/>
<xsl:template match="/Products">
<table>
<tr><th>Manufacturers</th></tr>
<xsl:for-each select="Maker">
<tr><td><xsl:value-of select="@name"/></td></tr>
</xsl:for-each>
</table>
</xsl:template>
</xsl:stylesheet>
''')
### END SOLUTION
p3a_xform = etree.XSLT(p3a_xslt)
p3a_out=p3a_xform(products)

#print("Output is", etree.tostring(p3a_out, encoding=str))
display(HTML(etree.tostring(p3a_out, encoding=str)))

Manufacturers
A
E
H


### 3b) An HTML table [ 5 pts ]

The table should include a row labeled Laptops, then print the model and price for each laptop. After that, it should print a row labeled PC and the model and price for each.

Your output should look like the following, but in HTML.

```
Laptops
Model	Price
2004	1150
2005	2500
2001	3673
PC
Model	Price
1001	2114
1002	995
1011	959
1012	649
```


In [62]:
### BEGIN SOLUTION
# I create a template matching on products
# Then I for-each on each Maker/Laptop
# I take the value-of the model attribute of the selected laptop
# And the value-of the price attribute of the selected Laptop
# I do the same for the Maker/Printer
p3b_xslt = etree.XML('''\
<xsl:stylesheet xmlns:xsl = "http://www.w3.org/1999/XSL/Transform">
<xsl:output method="text"/>
<xsl:template match="/Products">
<table>
<tr><td align="left">Laptops</td></tr>
<tr><td>
    <table>
        <tr><th>Model</th><th>Price</th></tr>
        <xsl:for-each select="Maker/Laptop">
        <tr>
            <td><xsl:value-of select="@model"/></td>
            <td><xsl:value-of select="@price"/></td>
        </tr>
        </xsl:for-each>
    </table>
</td></tr>
<tr><td align="left">PC</td></tr>
<tr><td>
    <table>
        <tr><th>Model</th><th>Price</th></tr>
        <xsl:for-each select="Maker/PC">
        <tr>
            <td><xsl:value-of select="@model"/></td>
            <td><xsl:value-of select="@price"/></td>
        </tr>
        </xsl:for-each>
    </table>
</td></tr>
</table>
</xsl:template>
</xsl:stylesheet>
''')
### END SOLUTION
p3b_xform = etree.XSLT(p3b_xslt)
p3b_out=p3b_xform(products)

#print("Output is", etree.tostring(p3b_out, encoding=str))
display(HTML(etree.tostring(p3b_out, encoding=str)))